In [2]:
import requests
from bs4 import BeautifulSoup
import json
import pandas as pd
import numpy as np
from py2neo import Graph,Node,Relationship
import pymysql

In [20]:
db = pymysql.connect("localhost","root","","med_zh" ,charset='utf8')
cursor = db.cursor()

In [21]:
med_zh = Graph(
    "http://localhost:7474", 
    username="neo4j", 
    password="bsoft"
)

In [22]:
def scrapy_triple(name):
    url = "http://www.tcminformatics.org/wiki/api.php?name={}".format(name)
    response = requests.get(url)
    contents = BeautifulSoup(response.content, 'lxml')
    dict_json = json.loads(contents.p.contents[0])
    links = pd.DataFrame(dict_json['links'])
    if (links.empty):
        return [pd.DataFrame(),pd.DataFrame()]
    nodes = pd.DataFrame(dict_json['nodes'])
    nodes['target']=range(len(nodes))
    merge = pd.merge(links,nodes,how ='outer', on ='target').drop(['name','value'],axis =1).sort_values(by='target')
    merge = pd.merge(merge,merge,how ='outer',left_on='source',right_on='target')
    merge = merge[merge['target_x'].notnull()].drop(['source_x','target_x','source_y','target_y','group_y'],axis =1 )
    merge['subject'] = merge[merge['group_x']=='1']['char2_x'][0]
    subject = merge[merge['group_x']=='1']
    subject = subject.fillna('中药')
    subject = subject.drop(['group_x'],axis=1)
    subject.columns = ['object', 'relation','subject']

    triples = merge[merge['group_x']=='3']
    triples = triples.drop(['group_x'],axis=1)
    triples.columns = ['object', 'relation','subject']
    triples = triples[triples['object'].notnull()]
#     print(url)
    return [subject,triples]

In [23]:
def subject_graph_saver(graph,subject):
    # test_node_1 = Node(label = "中药",name = "人参")
    subject = Node( subject['relation'][0],name = subject['subject'][0])
    graph.create(subject)
    return subject

In [24]:
def triple_graph_saver(graph,subject_node,triple):
    object_node = Node(triple['relation'],name = triple['object'])
#     print(object_node)
    graph.create(object_node)
    subject_relation_object = Relationship(subject_node,triple['relation'],object_node)
    graph.create(subject_relation_object)
#     print (subject_relation_object)

In [25]:
cursor.execute('select pk_id,name from terms')
datas = cursor.fetchall()
datas =[list(data) for data in datas]
all_triples =pd.DataFrame()
for data in datas:
    try:
        [subject,triples] = scrapy_triple(data[1])


    # #     print(data)
        if(triples.empty):
            continue
    #         datas[data[0]-1].append(0)
    #         cursor.execute('update terms set is_graph_exist=0 where pk_id={}'.format(str(data[0])))
        else:
    #         datas[data[0]-1].append(1)
            all_triples = all_triples.append(subject)
            all_triples = all_triples.append(triples)
            subject_node = subject_graph_saver(med_zh,subject)
            for index,triple in triples.iterrows():
                triple_graph_saver(med_zh,subject_node,triple)
    #         cursor.execute('update terms set is_graph_exist=1 where pk_id={}'.format(str(data[0])))
    finally:
        print(data,triples.empty)
cursor.close()

[1, '阿胶'] False
[2, '阿魏'] False
[3, '矮地茶'] True
[4, '艾片（左旋龙脑）'] True
[5, '艾叶'] False
[6, '安息香'] True
[7, '八角茴香'] False
[8, '巴豆'] False
[9, '巴豆霜'] False
[10, '巴戟天'] False
[11, '菝葜'] False
[12, '白扁豆'] False
[13, '白矾'] False
[14, '白附子'] False
[15, '白果'] False
[16, '白及'] False
[17, '白蔹'] False
[18, '白茅根'] False
[19, '白前'] False
[20, '白屈菜'] False
[21, '白芍'] False
[22, '白术'] False
[23, '白头翁'] False
[24, '白薇'] False
[25, '白鲜皮'] False
[26, '白芷'] False
[27, '百部'] False
[28, '百合'] False
[29, '柏子仁'] False
[30, '斑蝥'] False
[31, '板蓝根'] False
[32, '半夏'] False
[33, '半枝莲'] False
[34, '薄荷'] False
[35, '暴马子皮'] True
[36, '北豆根'] True
[37, '北刘寄奴'] True
[38, '北沙参'] False
[39, '荜茇'] True
[40, '荜澄茄'] False
[41, '蓖麻子'] False
[42, '萹蓄'] False
[43, '鳖甲'] False
[44, '槟榔'] False
[45, '冰片（合成龙脑）'] True
[46, '补骨脂'] False
[47, '布渣叶'] True
[48, '苍耳子'] False
[49, '苍术'] False
[50, '藏菖蒲'] False
[51, '草豆蔻 Caodoukou'] True
[52, '草果'] False
[53, '草乌'] False
[54, '草乌叶'] False
[55, '侧柏叶'] False
[56, '蝉蜕'] True
[57, '蟾酥'] False

[449, '使君子'] False
[450, '柿蒂'] False
[451, '首乌藤'] True
[452, '熟地黄'] False
[453, '水飞蓟'] False
[454, '水红花子'] False
[455, '水牛角'] False
[456, '水蛭'] False
[457, '丝瓜络'] False
[458, '四季青'] True
[459, '松花粉'] False
[460, '苏合香'] False
[461, '苏木'] False
[462, '酸枣仁'] False
[463, '娑罗子'] False
[464, '锁阳'] False
[465, '太子参'] False
[466, '檀香'] False
[467, '桃仁'] False
[468, '桃枝'] False
[469, '体外培育牛黄'] True
[470, '天冬'] True
[471, '天花粉'] False
[472, '天葵子'] False
[473, '天麻'] False
[474, '天南星'] False
[475, '天然冰片（右旋龙脑）'] True
[476, '天山雪莲'] True
[477, '天仙藤'] False
[478, '天仙子'] False
[479, '天竺黄'] True
[480, '甜瓜子'] False
[481, '铁皮石斛'] False
[482, '葶苈子'] False
[483, '通草'] False
[484, '通关藤'] False
[485, '土贝母'] False
[486, '土鳖虫（蜇虫）'] True
[487, '土茯苓'] False
[488, '土荆皮'] False
[489, '土木香'] False
[490, '菟丝子'] False
[491, '瓦楞子'] False
[492, '瓦松'] False
[493, '王不留行'] False
[494, '威灵仙'] False
[495, '委陵菜'] False
[496, '乌梅'] False
[497, '乌梢蛇'] False
[498, '乌药'] False
[499, '巫山淫羊藿'] True
[500, '吴茱萸'] False
[501, '蜈蚣'] Fa

In [53]:
[subject,triples] = scrapy_triple('绿矾')

In [54]:
subject

""


In [38]:
triples

""


In [16]:
subject.append(triples)

,object,relation,subject
0,冬葵果,中药,冬葵果
5,冬葵果,中文正名,冬葵果
6,加木巴,中文异名,冬葵果
7,活血药,上位词,冬葵果
8,锦葵科,上位词,冬葵果
9,调经药,上位词,冬葵果
10,中药,类型,冬葵果


In [33]:
triples[triples['object'].notnull()]

,object,relation,subject
6,毕茄,中文异名,荜澄茄
7,山苍子,中文异名,荜澄茄
8,毗陵茄子,中文异名,荜澄茄
9,山鸡椒,中文异名,荜澄茄
10,澄茄,中文异名,荜澄茄
11,毕澄茄,中文异名,荜澄茄
12,"Mountain Spicy Tree Fruit, Fruit of Mountain S...",英文异名,荜澄茄
13,Fructus Litseae,英文异名,荜澄茄
14,荜澄茄,中文正名,荜澄茄
15,祛风药,上位词,荜澄茄


In [32]:
triples['subject']

6     荜澄茄
7     荜澄茄
8     荜澄茄
9     荜澄茄
10    荜澄茄
11    荜澄茄
12    荜澄茄
13    荜澄茄
14    荜澄茄
15    荜澄茄
16    荜澄茄
17    荜澄茄
18    荜澄茄
19    荜澄茄
Name: subject, dtype: object

In [27]:
all_triples.to_csv('zh_med.csv')